In [54]:
import numpy as np
from sklearn import datasets
np.random.seed(2042)

In [79]:
iris = datasets.load_iris()
X = iris.data[:,(2,3)]
y = iris.target

In [56]:
X_bias = np.c_[np.ones((len(X),1)),X]

In [57]:
def train_test_val_split(X,y,shuffle=True,test_ratio=0.2,val_ratio=0.2):
    size = len(X)
    if shuffle:
        indexes = np.random.permutation(len(X))
    else:
        indexes = range(len(X))
    X_ = X[indexes]
    y_ = X[indexes]

    test_size = int(size*test_ratio)
    val_size = int(test_size*val_ratio)
    train_size = size-test_size-val_size

    X_train = X_[:train_size]
    y_train = y_[:train_size]
    X_val = X_[train_size:-test_size]
    y_val = y_[train_size:-test_size]
    X_test = X_[-test_size:]
    y_test = y_[-test_size:]

    return X_train,y_train,X_val,y_val,X_test,y_test

In [58]:
X_train,y_train,X_val,y_val,X_test,y_test = train_test_val_split(X_bias,y)

In [116]:
def hot_encode(y):
    num_classes = len(np.unique(y))
    m = len(y)
    y_hot_encode = np.zeros((m,num_classes))
    y_hot_encode[np.arange(m) ,y] = 1 
    return y_hot_encode

In [75]:
def score(X,Theta):
    return X.dot(Theta)

In [60]:
def softmax_function(scores):
    dividend = np.exp(scores)
    divisor = np.sum(dividend, axis=1, keepdims=True)
    return dividend/divisor

In [66]:
def cross_entropy_gradient(X,y,probabilities):
    m = len(X)
    return (1/m)*np.sum((probabilities-y),axis=0,keepdims=True)

In [83]:
num_features = X_bias.shape[1] 
num_classes = len(np.unique(y))

In [86]:
probabilities.shape

(150, 3)

In [84]:
scores = score(X_bias,np.random.rand(num_features,num_classes))
probabilities = softmax_function(scores)
gradient = cross_entropy_gradient(X_bias,y,probabilities)

ValueError: operands could not be broadcast together with shapes (150,) (150,3) 

In [72]:
scores.shape

(150, 150)

In [63]:
np.random.rand(X_bias.shape[0],X_bias.shape[1]).T

array([[0.26230022, 0.63436278, 0.05938469, 0.95103372, 0.73343737,
        0.79737699, 0.47744231, 0.23538208, 0.60729814, 0.48213886,
        0.06319302, 0.93431257, 0.20844087, 0.62198934, 0.96468795,
        0.89792467, 0.76543243, 0.02138724, 0.0300963 , 0.57364873,
        0.76182301, 0.3948555 , 0.89189935, 0.10764505, 0.60370226,
        0.20446886, 0.65952021, 0.53170934, 0.53682797, 0.7402216 ,
        0.61979949, 0.34956195, 0.33342459, 0.80807363, 0.50012603,
        0.02832748, 0.56693008, 0.99117108, 0.71052899, 0.94179716,
        0.62104826, 0.26345465, 0.81130808, 0.06866469, 0.37584373,
        0.72777915, 0.33889488, 0.72453189, 0.53326416, 0.83095051,
        0.66773076, 0.25161212, 0.86237103, 0.5861179 , 0.33646903,
        0.05314185, 0.10935901, 0.69098309, 0.55646942, 0.26404435,
        0.65327914, 0.90999003, 0.77078428, 0.15902364, 0.83079051,
        0.08147389, 0.7669885 , 0.49866429, 0.51139385, 0.92649505,
        0.53968993, 0.19878942, 0.58829653, 0.32